
# Gpytorch / Pylightcurve-torch tutorial
Gpytorch is ...

GPs for exo-transits ...

Pros of using GPytorch:

- Exact or approximate GPs
- Deep GPs
- Integration of Neural Nets moduls with Pytorch
- Scalable, flexible...

In this notebook, simple examples of use of GP along with Pylightcurve-torch transit module.
Examples adapted from [GPytorch tutorials](https://docs.gpytorch.ai/en/v1.1.1/examples/01_Exact_GPs/Simple_GP_Regression.html)

# TODO:

- Proper Parameters (hyperparameters) estimation ! 
- solve the nans problem with kepler data --> solved, but should we introduce constraints?
- Run on GPUs
- Scaling, how does that work for long LCs?
- which params to fit? (not really the quetion here but...)

In [ ]:
import math
import numpy as np
import torch
import gpytorch
import matplotlib.pylab as plt

from pylightcurve_torch import TransitModule

%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams['font.size'] = 14

# Toy Light Curve

### Generate data

In [ ]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 200).double().detach()  # Time vector

# Transit model
pars =  {'e':0.01, 'i':90., 'w':0., 'rp': 0.1, 'method': 'linear', 'ldc': [0.1], 
             'P': 4., 't0': 0.6, 'a': 7., 'fp': 0.0001}
tm = TransitModule(train_x, **pars, )

# True function is sin(2*pi*x) with Gaussian noise and transit
train_data = 100. + (torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * np.sqrt(0.01)) / 2
train_data *= tm()[0]

plt.figure(figsize=(12,4))
plt.plot(train_x, train_data.detach())
plt.title('Data to fit')
plt.ylabel('Flux')
plt.xlabel('Time (days)')

# Standardise
mu = train_data.mean()
sigma = train_data.std()

# mu = 0
# sigma = 1

preprocess = lambda y: (y - mu) / sigma
postprocess = lambda y: y * sigma + mu
# preprocess = lambda y: y
train_y = preprocess(train_data)
train_y.double().detach_()
pass


### Define GP model

In [ ]:

tm.set_params(**pars)
tm.set_params(rp=0.2, t0=0.55)
tm.activate_grad('rp','t0')   # which transit params to fit

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, tm):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean() 
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.transit_module = tm 

    def forward(self, x):
        mean_x = (mu + self.mean_module(x)) 
        mean_x *= self.transit_module(time=x.T)[0]  # Careful to the input here, time has to be last dim not first
        mean_x = preprocess(mean_x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

        
    
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood().double()
model = ExactGPModel(train_x, train_y, likelihood, tm).double()

In [ ]:
training_iter = 100

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': list(model.mean_module.parameters()) 
               + list(model.covar_module.parameters())
               + list(model.likelihood.parameters())},
    {'params': list(model.transit_module.parameters()), 'lr':0.01},  #### different LR for transit par
], lr=0.1)  # Default LR?

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


# Optimise the GP model
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.5f  rp: %.5f  t0: %.5f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item(),
        model.transit_module.rp.item(),
        model.transit_module.t0.item()
    ))
    optimizer.step()

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
#     test_x = torch.linspace(0, 1, 301).double()
    test_x = train_x 
    observed_pred = likelihood(model(test_x))

In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(4, 1, figsize=(12, 12), sharex=True)
    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax[0].plot(train_x.numpy(), train_y.numpy(), 'k*', label='Observed Data')
    # Plot predictive means as blue line
#     ax[0].plot(test_x.numpy(), preprocess(postprocess(observed_pred.mean) / model.transit_module(time=test_x)[0]), 'r', label='stellar component of the mean')
    ax[0].plot(test_x.numpy(), observed_pred.mean.numpy(), 'b', label='Mean')
    # Shade between the lower and upper confidence bounds
    ax[0].fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5, label='Confidence')
    #ax.set_ylim([-3, 3])
    ax[0].set_ylabel('normalised flux')
    ax[0].set_title('Fitted data and model')
    ax[0].legend()
    
    ax[1].plot(test_x.numpy(), postprocess(observed_pred.mean) / model.transit_module(time=test_x)[0])
    ax[1].set_title('Stellar component')
    ax[1].set_ylabel('Flux')

    ax[2].plot(test_x.numpy(), model.transit_module(time=test_x)[0].detach())
    ax[2].set_title('Transit model')
    ax[2].set_ylabel('stellar flux units')
    
    ax[3].scatter(test_x.numpy(), postprocess(train_y - observed_pred.mean))
    ax[3].set_title('Residuals')
    ax[3].set_ylabel('Flux')
    ax[3].set_xlabel('Time (days)')
    
    
    plt.tight_layout()

# Tess Light Curve

What's the point here: scalability? fitting more params? real data? 

Section could well be ditched otherwise, for simplicity

In [ ]:
import lightkurve as lk
lc_search = lk.search_lightcurve('kepler-1b')[0]
lc = lc_search.download()
lc.plot()

In [ ]:
lc = lc[(lc.time.value>122.) & (lc.time.value<126.)]
lc.plot()

In [ ]:
train_x = torch.tensor(lc.time.value.astype(np.float64))
train_y = torch.tensor(lc.flux.value.astype(np.float64))

# Transit model
pars =  {'e':0., 'i':83.8, 'w':0., 'rp': 0.2, 'method': 'linear', 'ldc': [0.1], 
         'P': 2.4706132, 't0': 122.71, 'a': 7.9, 'fp': 0.0001}

tm = TransitModule(train_x, **pars)

# med_flux = np.nanmedian(train_y.detach().numpy())

mu = np.nanmean(train_y)
sigma = np.nanstd(train_y)
preprocess = lambda y: (y - mu) / sigma
postprocess = lambda y: y * sigma + mu

train_y = preprocess(train_y)
train_y[np.isnan(train_y)] = 0. 

In [ ]:

tm.set_params(**pars)
tm.set_params(rp=0.2, t0=122.75) #, a=30, ldc=0.2)
tm.activate_grad('rp','t0', 'P')   # which transit params to fit

In [ ]:

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, tm):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean() 
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.transit_module = tm 

    def forward(self, x):
        mean_x = (mu + self.mean_module(x)) 
        mean_x *= self.transit_module(time=x.T)[0]  # Careful to the input here, time has to be last dim not first
        mean_x = preprocess(mean_x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood().double()
model = ExactGPModel(train_x, train_y, likelihood, tm).double()

In [ ]:
training_iter = 50

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': list(model.mean_module.parameters()) 
               + list(model.covar_module.parameters())
               + list(model.likelihood.parameters())},
    {'params': list(model.transit_module.parameters()), 'lr':0.005},  #### different LR for transit par
], lr=0.01)  # Default LR?

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Optimise the GP model
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.5f  rp: %.5f  t0: %.5f P: %.5f  ' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item(),
        model.transit_module.rp.item(),
        model.transit_module.t0.item(),
        model.transit_module.P.item(),
#         model.transit_module.ldc.item()
    ))
    optimizer.step()

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(train_x[0], train_x[-1], 301).double()
    test_x = train_x
    observed_pred = likelihood(model(test_x))

In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(4, 1, figsize=(12, 12), sharex=True)
    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax[0].plot(train_x.numpy(), train_y.numpy(), 'k*', label='Observed Data')
    # Plot predictive means as blue line
#     ax[0].plot(test_x.numpy(), preprocess(postprocess(observed_pred.mean) / model.transit_module(time=test_x)[0]), 'r', label='stellar component of the mean')
    ax[0].plot(test_x.numpy(), observed_pred.mean.numpy(), 'b', label='Mean')
    # Shade between the lower and upper confidence bounds
    ax[0].fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5, label='Confidence')
    #ax.set_ylim([-3, 3])
    ax[0].set_ylabel('normalised flux')
    ax[0].legend()
    
    ax[1].plot(test_x.numpy(), postprocess(observed_pred.mean) / model.transit_module(time=test_x)[0])
    ax[1].set_title('Stellar component')
    ax[1].set_ylabel('Flux')

    ax[2].plot(test_x.numpy(), model.transit_module(time=test_x)[0].detach())
    ax[2].set_title('Transit model')
    ax[2].set_ylabel('stellar flux units')
    
    ax[3].scatter(test_x.numpy(), postprocess(train_y - observed_pred.mean), s=3)
    ax[3].set_title('Residuals')
    ax[3].set_ylabel('Flux')
    ax[3].set_xlabel('Time (days)')